# Desarrollo de análisis

En este notebook se presenta el tratamiento de la base escogida de CIC-DDoS2019, divido en los siguientes puntos:

1. **Cargado de base de datos:** Carga de la base de datos escogida, previamente transformada y en formato parquet.
2. **Análisis univariado**: Identificación de formato del dataset y de columnas que no brindan información (aquellas con el mismo valor en todos los registros).
2. **Análisis de Outliers:** Filtrado de valores atípicos mediante el análisis del [rango intercuartil](https://www.geeksforgeeks.org/interquartile-range-to-detect-outliers-in-data/). Finalmente al descartar más de 3/4 partes de la base original, se decidió quedarse con la base inicial.
3. **Eliminación de elementos correlacionados:** Revisión de elementos correlacionados, basada en el nombre de las variables y en el análisis previo de la composición de la base de datos.
4. **Transformación de etiquetas:** Para la predicción, se realizó la homogeneización de etiquetas, ya que varias de ellas se referían a los mismos ataques.
5. **Guardado de versión ajustada:** Guardado de dataset transformado para el proceso de predicción

# 1. Cargado de base

In [ ]:
# Montado de drive para realizar el análisis
from google.colab import drive
drive.mount('/content/drive') #Comentar
#Librerías Utilizadas
import pandas as pd # Manipulación de datos
pd.options.display.float_format = '{:.1f}'.format #Adecuar formato de salidas de tablas a 1 posición decimal
import seaborn as sns # Creación de gráficas
import matplotlib.pyplot as plt # Formato de gráficas
import numpy as np # Tratamiento de información
pd.options.mode.chained_assignment = None # Eliminar warnings de pandas por reemplazo de información sin .loc

Mounted at /content/drive


In [ ]:
# Cargado de información
df_CIC2019 = pd.read_parquet("/content/drive/MyDrive/Tesis - Machine Learning/Tesis - Machine Learning/B - bases de datos/Análisis Abril 2024/Bases Consolidadas/Total_CICDDoS2019.parquet")

In [ ]:
df_CIC2019

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,17,48,4,0,1616.0,0.0,440.0,368.0,404.0,41.6,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
1,17,557,112,0,49280.0,0.0,440.0,440.0,440.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
2,6,323228,14,12,1736.0,526.0,602.0,0.0,124.0,211.3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,17,46917,2,2,86.0,118.0,43.0,43.0,43.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,17,254,58,0,25520.0,0.0,440.0,440.0,440.0,0.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431366,17,2001326,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
431367,17,355,46,0,20096.0,0.0,440.0,368.0,436.9,14.8,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
431368,17,3002731,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
431369,17,256,52,0,22880.0,0.0,440.0,440.0,440.0,0.0,...,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP


# 2. Análisis Univariado

En primera instancia, se obtuvo el total de filas para realizar el análisis univariado:

In [ ]:
total, _ = df_CIC2019.shape

Posteriormente, a partir del siguiente código se identifica que no hay ninguna columna con valores nulos de la cual sea necesario hacer un tratamiento adicional de la información:

In [ ]:
df_CIC2019.isna().sum().sort_values(ascending=False)

Protocol                0
CWE Flag Count          0
Fwd Avg Packets/Bulk    0
Fwd Avg Bytes/Bulk      0
Avg Bwd Segment Size    0
                       ..
Bwd IAT Total           0
Fwd IAT Min             0
Fwd IAT Max             0
Fwd IAT Std             0
Label                   0
Length: 78, dtype: int64

Con esto en mente, se calcula la moda de cada una de las columnas y se filtra para identificar su frecuencia. Si la frecuencia del valor de moda corresponde con el total de registros, significa que todos los valores son los mismos.

Por ello, se crea una lista con las columnas a eliminar, ya que no aportarían información adicional al modelo:

In [ ]:
#Código para ver variables que sólo tengan un valor
drop_uniq=[]
for i in df_CIC2019.columns:
  # Cálculo de moda
  moda=df_CIC2019[i].mode()[0]
  # Conteo de registros
  conteo=df_CIC2019[df_CIC2019[i]==moda][i].count()
  if conteo == total:
    print(f"La columna {i} tiene el mismo valor en todos los registros")
    # Guardado en lista única
    drop_uniq.append(i)
print(drop_uniq)

La columna Bwd PSH Flags tiene el mismo valor en todos los registros
La columna Fwd URG Flags tiene el mismo valor en todos los registros
La columna Bwd URG Flags tiene el mismo valor en todos los registros
La columna FIN Flag Count tiene el mismo valor en todos los registros
La columna PSH Flag Count tiene el mismo valor en todos los registros
La columna ECE Flag Count tiene el mismo valor en todos los registros
La columna Fwd Avg Bytes/Bulk tiene el mismo valor en todos los registros
La columna Fwd Avg Packets/Bulk tiene el mismo valor en todos los registros
La columna Fwd Avg Bulk Rate tiene el mismo valor en todos los registros
La columna Bwd Avg Bytes/Bulk tiene el mismo valor en todos los registros
La columna Bwd Avg Packets/Bulk tiene el mismo valor en todos los registros
La columna Bwd Avg Bulk Rate tiene el mismo valor en todos los registros
['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'FIN Flag Count', 'PSH Flag Count', 'ECE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg 

Esto se rectifica con la función `.describe()`, mostrando que todos los valores de las columnas seleccionadas son igual a 0

In [ ]:
df_CIC2019[drop_uniq].describe()

,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,FIN Flag Count,PSH Flag Count,ECE Flag Count,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate
count,431371.0,431371.0,431371.0,431371.0,431371.0,431371.0,431371.0,431371.0,431371.0,431371.0,431371.0,431371.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Al hacer la eliminación de duplicados, se reduce el total de columnas a 66:

In [ ]:
df_CIC2019_filt=df_CIC2019.drop(columns=drop_uniq)
df_CIC2019_filt

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,17,48,4,0,1616.0,0.0,440.0,368.0,404.0,41.6,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
1,17,557,112,0,49280.0,0.0,440.0,440.0,440.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
2,6,323228,14,12,1736.0,526.0,602.0,0.0,124.0,211.3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,17,46917,2,2,86.0,118.0,43.0,43.0,43.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,17,254,58,0,25520.0,0.0,440.0,440.0,440.0,0.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431366,17,2001326,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
431367,17,355,46,0,20096.0,0.0,440.0,368.0,436.9,14.8,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
431368,17,3002731,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
431369,17,256,52,0,22880.0,0.0,440.0,440.0,440.0,0.0,...,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP


# 3. Análisis de Outliers

Con el objetivo de reducir todos los valores atípicos de las columnas, se implementó el siguiente for, en el que se aplica el método de eliminación de valores atípicos utilizando el rango intercuartil. De esta manera, se va filtrando el dataframe de referencia:

In [ ]:
from numpy import percentile #Cálculo de percentil a partir de numpy
df_CIC_out=df_CIC2019_filt.copy()
for i in df_CIC2019_filt.columns:
  if i != "Label":
    # Cálculo de rango intercuartil
    q25 = percentile(df_CIC2019_filt[i],25)
    q75 = percentile(df_CIC2019_filt[i],75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower , upper = q25 - cut_off, q75 + cut_off
    # Filtro de dataframe
    df_CIC_out=df_CIC_out[(df_CIC_out[i] >= lower) & (df_CIC_out[i] <= upper)]

Se identificó que el número de registros se reduce a menos de 100 mil. Por lo tanto, en este caso se optará por mantener toda la información original y se procederá a realizar la respectiva transformación de las variables para su uso en los modelos.

In [ ]:
df_CIC_out

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
8,17,643,12,0,5280.0,0.0,440.0,440.0,440.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
10,17,2996975,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
21,17,3002291,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
22,17,3006969,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
26,17,3002006,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431344,17,3007760,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
431350,17,8392,10,0,4400.0,0.0,440.0,440.0,440.0,0.0,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
431364,17,6137890,6,0,3096.0,0.0,516.0,516.0,516.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
431366,17,2001326,4,0,2064.0,0.0,516.0,516.0,516.0,0.0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP


# 4. Eliminación de elementos correlacionados

Para este análisis, se realizó el tratamiento de las columnas con el objetivo de identificar aquellas que corresponden a una misma variable. Se implementó el siguiente código, en el cual se reemplazan las principales estadísticas (Max, Min, etc.) por un valor vacío:

In [ ]:
count_val=[i.replace(" Std","").replace(" Max","").replace(" Min","").replace("Mean","").replace("Variance","").strip() for i in df_CIC2019_filt.columns]

Con esto, se hace un conteo de los valores únicos a partir de la creación de un dataframe:

In [ ]:
# Creación de dataframe
df_cols=pd.DataFrame(count_val)
#Referencia para conteo
df_cols["count"]=1
df_cols.columns=["Name","ref"]
# Agrupación y conteo
df_cols=df_cols.groupby(by="Name").count().sort_values(by="ref",ascending=False).reset_index()
df_cols.head(10)

,Name,ref
0,Packet Length,5
1,Fwd Packet Length,4
2,Active,4
3,Fwd IAT,4
4,Idle,4
5,Bwd IAT,4
6,Bwd Packet Length,4
7,Flow IAT,4
8,Fwd Seg Size,1
9,Init Bwd Win Bytes,1


Se identifica que hay 8 variables que tienen más de 1 métrica en el Dataset, y que de acuerdo con análisis previos muestran una alta correlación:

In [ ]:
# Lista de variables con más de 2 columnas
other_cols=list(df_cols[df_cols["ref"]>1]["Name"])
other_cols

['Packet Length',
 'Fwd Packet Length',
 'Active',
 'Fwd IAT',
 'Idle',
 'Bwd IAT',
 'Bwd Packet Length',
 'Flow IAT']

Con ello, se procede a escoger la media de estas columnas en conjunto con las ya existentes utilizando List Comprehension:

In [ ]:
# Escoger columnas que no tienen las métricas
final_cols=[item for item in df_CIC2019_filt.columns if not any(item.startswith(palabra) for palabra in other_cols)]
# Para las métricas sólo escoger las que corresponden a la media
mean_cols=[i + " Mean" for i in other_cols ]

In [ ]:
# Total de columnas
Col_filter=final_cols + mean_cols

A partir de esto, se obtiene un dataframe de 39 columnas, que se usará para el desarrollo de modelos:

In [ ]:
df_CIC2019_corr=df_CIC2019_filt[final_cols + mean_cols]
df_CIC2019_corr.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Flow Bytes/s,Flow Packets/s,Fwd PSH Flags,Fwd Header Length,...,Fwd Seg Size Min,Label,Packet Length Mean,Fwd Packet Length Mean,Active Mean,Fwd IAT Mean,Idle Mean,Bwd IAT Mean,Bwd Packet Length Mean,Flow IAT Mean
0,17,48,4,0,1616.0,0.0,33666666.7,83333.3,0,80,...,20,DrDoS_NTP,411.2,404.0,0.0,16.0,0.0,0.0,0.0,16.0
1,17,557,112,0,49280.0,0.0,88473967.7,201077.2,0,0,...,0,DrDoS_NTP,440.0,440.0,0.0,5.0,0.0,0.0,0.0,5.0
2,6,323228,14,12,1736.0,526.0,6998.2,80.4,0,304,...,20,Benign,83.8,124.0,0.0,24863.7,0.0,15038.7,43.8,12929.1
3,17,46917,2,2,86.0,118.0,4348.1,85.3,0,40,...,20,Benign,49.4,43.0,0.0,3.0,0.0,2.0,59.0,15639.0
4,17,254,58,0,25520.0,0.0,100472440.9,228346.5,0,-58,...,-1,DrDoS_NTP,440.0,440.0,0.0,4.5,0.0,0.0,0.0,4.5


# 5. Transformación de etiquetas

Posteriormente, se agruparon las etiquetas de ataques similares entre si, principalmente para LDAP, UDP, NetBIOS y MSSQL:


In [ ]:
# LDAP
lista_LDAP= ["DrDoS_LDAP"]
df_CIC2019_corr['Label'] = df_CIC2019_corr['Label'].apply(lambda x: 'LDAP' if x in lista_LDAP else x)
# UDP
lista_UDP= ["DrDoS_UDP","UDP-lag","UDPLag"]
df_CIC2019_corr['Label'] = df_CIC2019_corr['Label'].apply(lambda x: 'UDP' if x in lista_UDP else x)
# NetBIOS
lista_NetBIOS= ["DrDoS_NetBIOS"]
df_CIC2019_corr['Label'] = df_CIC2019_corr['Label'].apply(lambda x: 'NetBIOS' if x in lista_NetBIOS else x)
# MSSQL
lista_MSSQL= ["DrDoS_MSSQL"]
df_CIC2019_corr['Label'] = df_CIC2019_corr['Label'].apply(lambda x: 'MSSQL' if x in lista_MSSQL else x)

# 6. Guardado de versión ajustada

Finalmente, se hace el guardado de la versión actualizada:

In [ ]:
# Guardado de archivo
df_CIC2019_corr.to_parquet("CICIDS2019_2.parquet")